In [1]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 KB 789.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 KB 10.8 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.2 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install faiss-gpu

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:01m
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("01bool.pdf")
pages = loader.load_and_split()

In [4]:
pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 KB 373.1 kB/s eta 0:00:00 kB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Define our text splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=64, separator=".")
docs = text_splitter.split_documents(pages)

In [7]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_******************************"

In [8]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.environ["HUGGINGFACEHUB_API_TOKEN"], model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [9]:
from langchain.vectorstores import FAISS

faiss_index = FAISS.from_documents(docs, embeddings)

In [10]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("human", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say this exact sentence: The question you are asking does not fall into our Knowledge base. Use about 200 words and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"),
    ]
)

In [11]:
chat_template

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say this exact sentence: The question you are asking does not fall into our Knowledge base. Use about 200 words and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [14]:
from langchain.llms import HuggingFaceHub, HuggingFacePipeline

repo_id = 'mistralai/Mistral-7B-Instruct-v0.1'

llm = HuggingFaceHub(
    repo_id=repo_id, 
    model_kwargs={"temperature": 0.5, "max_length": 1024}
)

/home/subash/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/subash/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [13]:
pip install huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 KB 189.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 KB 324.9 kB/s eta 0:00:001m331.4 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 343.2 kB/s eta 0:00:001m415.3 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

In [16]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=faiss_index.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": chat_template},
    memory=memory
)

In [18]:
question = "What is boolean retrieval?"
qa_chain({"query": question})

{'query': 'What is boolean retrieval?',
 'chat_history': [SystemMessage(content='\nThe human asks what boolean retrieval is. The AI explains that boolean retrieval is a model for information retrieval in which the query is in the form of a Boolean expression of terms, combined with the operators AND, OR, and NOT. The model views each document as a set of words, and the retrieval process is based on the proximity of the terms in the documents. This type of retrieval was the main or only search option provided by large commercial information providers for three decades')],
 'result': ' Boolean retrieval is a model for information retrieval in which the query is in the form of a Boolean expression of terms, combined with the operators AND, OR, and NOT. The model views each document as a set of words, and the retrieval process is based on the proximity of the terms in the documents. This type of retrieval was the main or only search option provided by large commercial information providers

In [19]:
question = "Tell me more in detail"
qa_chain({"query": question})

{'query': 'Tell me more in detail',
 'chat_history': [SystemMessage(content='\nThe human asks what boolean retrieval is. The AI explains that boolean retrieval is a model for information retrieval in which the query is in the form of a Boolean expression of terms, combined with the operators AND, OR, and NOT. The model views each document as a set of words, and the retrieval process is based on the proximity of the terms in the documents. This type of retrieval was the main or only search option provided by large commercial information providers for three decades')],
 'result': ' The article of Bush (1945) introduced the concept of a personal information retrieval system, known as "memex," which is a device that stores an individual\'s books, records, and communications and allows for quick and flexible access to them. This device was intended to serve as an enlarged intimate supplement to one\'s memory. In contrast, modern information retrieval systems operate on a much larger scale, 

In [20]:
question = "Tell me about Elon Musk's contribution in Neural Networks"
qa_chain({"query": question})

{'query': "Tell me about Elon Musk's contribution in Neural Networks",
 'chat_history': [SystemMessage(content='\nThe human asks what boolean retrieval is. The AI explains that boolean retrieval is a model for information retrieval in which the query is in the form of a Boolean expression of terms, combined with the operators AND, OR, and NOT. The model views each document as a set of words, and the retrieval process is based on the proximity of the terms in the documents. This type of retrieval was the main or only search option provided by large commercial information providers for three decades')],
 'result': ' Elon Musk did not make any significant contribution in the field of Neural Networks.'}